In [1]:
!guardrails hub install hub://guardrails/two_words --quiet

Installing hub://guardrails/two_words...
✅Successfully installed guardrails/two_words!




## Input Validation

Guardrails supports validating inputs (prompts, instructions, msg_history) with string validators.

In XML, specify the validators on the `prompt` or `instructions` tag, as such:

In [2]:
from guardrails import Guard

rail_spec = """
<rail version="0.1">
<prompt
    validators="hub://guardrails/two_words"
    on-fail-two-words="exception"
>
This is not two words
</prompt>
<output type="string">
</output>
</rail>
"""

guard = Guard.from_rail_string(rail_spec)

When `fix` is specified as the on-fail handler, the prompt will automatically be amended before calling the LLM.

In any other case (for example, `exception`), a `ValidationError` will be returned in the outcome.

In [4]:
import os
from guardrails.errors import ValidationError

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

try:
    guard(
        model="gpt-4o"
    )
except ValidationError as e:
    print(e)

When using pydantic to initialize a `Guard`, input validators can be specified by composition, as such:

In [5]:
from guardrails.hub import TwoWords
from pydantic import BaseModel


class Pet(BaseModel):
    name: str
    age: int


guard = Guard.from_pydantic(Pet)
guard.use(TwoWords(on_fail="exception"), on="prompt")

try:
    guard(
        model="gpt-4o",
        prompt="This is not two words",
    )
except ValidationError as e:
    print(e)

Validation failed for field with errors: Value must be exactly two words
